# Genetic Algotihm in Decision Tree

In [1]:
import numpy as np
import pandas as pd
import geneticDicisionTree
from sklearn.tree import DecisionTreeClassifier

In [2]:
dataset = pd.read_csv('dataset\ProjectK.csv')

In [3]:
# Commit_age를 기준으로 정렬 및 인덱스 재설정
s=dataset.sort_values(by=['commit_age'],ascending= True,axis=0)
df2 = s.reset_index(drop=True)

In [4]:
split_strandar_index = 941

In [5]:
df2_train = df2[:split_strandar_index]
df2_test =df2[split_strandar_index:]
print(df2_train['bug'].value_counts())
print(df2_test['bug'].value_counts())

False    764
True     177
Name: bug, dtype: int64
False    1376
True      198
Name: bug, dtype: int64


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df2_test)
scaler.transform(df2_train)

array([[ 1.04498079, -0.45024306, -0.35889597, ..., -2.53126456,
        -4.00332634, -0.3793354 ],
       [-0.49943213, -0.54318835, -0.59339254, ..., -2.53126456,
        -2.2752163 , -0.3793354 ],
       [ 1.81718724, -0.54633903, -0.59339254, ..., -2.53126456,
        -1.54850555, -0.3793354 ],
       ...,
       [-0.49943213, -0.54318835, -0.59339254, ..., -1.43918145,
        -0.35081822, -0.3793354 ],
       [ 1.04498079, -0.55421575, -0.59339254, ..., -0.56551496,
        -0.25337472, -0.3793354 ],
       [-0.49943213, -0.32106487, -0.47614426, ..., -1.29357037,
        -0.34975418,  2.63618948]])

In [7]:
X_train = np.array(df2_train.loc[:,'author':'commitdate'])
y_train = np.array(df2_train['bug'])

X_test = np.array(df2_test.loc[:,'author':'commitdate'])
y_test = np.array(df2_test['bug'])
X_train.shape

(941, 12)

In [8]:
split_train_vali = int(X_train.shape[0]*0.7)
X_g_train = X_train[:split_train_vali]
X_g_vali = X_train[split_train_vali:]
y_g_train = y_train[:split_train_vali]
y_g_vali = y_train[split_train_vali:]
print(X_g_train.shape)
print(X_g_vali.shape)

(658, 12)
(283, 12)


In [ ]:
import Implementation
import g_measure
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, matthews_corrcoef, confusion_matrix, classification_report, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

g_measure_score = []
best_fitness=[]
accuracy=[]
best_gene= np.empty([7, 1])
conf_matrix_list_of_arrays = []
class_report_list_of_arrays = []
roc_auc_score_list_of_arrays = []
mcc_list_of_arrays = []
cm = None
y_predicted_overall = None
y_test_overall = None  
for i in range(10):
    print('--------------------------%s 번째 Train--------------------------'%(i+1))
    sm = SMOTE(random_state=42)
    X_g_train, y_g_train = sm.fit_resample(X_g_train, y_g_train)
    fitness, best_gene = Implementation.generation(X_train=X_g_train,
                                                   X_vali=X_g_vali,
                                                   y_train=y_g_train,
                                                   y_vali=y_g_vali)
        
    balance = {True:best_gene[5],False:best_gene[6]}
        
    if best_gene[0]==0:
            cr='gini'
    else:
            cr='entropy'
            
    clf = DecisionTreeClassifier(criterion=cr,
                                max_depth=int(best_gene[1]),
                                min_samples_split=int(best_gene[2]),
                                max_leaf_nodes=int(best_gene[3]),
                                random_state=int(best_gene[4]),
                                class_weight=balance
                                     )
    X_train, y_train = sm.fit_resample(X_train, y_train)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
        
    clf_score = accuracy_score(pred, y_test)
    accuracy.append(clf_score)
    g_score = g_measure.score(y_test,pred)
    print('\n***%s 번째 G_measure = %s***\n'%(i+1,g_score))
    g_measure_score.append(g_score)
    conf_matrix = confusion_matrix(y_test, pred)
    conf_matrix_list_of_arrays.append(conf_matrix)
    roc_auc = roc_auc_score(y_test, pred)
    roc_auc_score_list_of_arrays.append(roc_auc)
    mcc = matthews_corrcoef(y_test, pred)
    mcc_list_of_arrays.append(mcc)
    cv_cm = confusion_matrix(y_test, pred)
    # sum the cv per fold
    if cm is None:
        cm = cv_cm
    else:
        cm += cv_cm
    print(classification_report(y_test, pred, digits=3))
    print(cm)


print('Accuracy : ', accuracy)
print('\n')
print('G-measure : ', g_measure_score)
print('\n')
print('Mean G-measure : ', np.array(g_measure_score).mean())

--------------------------1 번째 Train--------------------------
Initilialize Poplulation------> Complete



-------------------------------------------------------
Best parameters are:
criterion gini
max_depth 14.0
min_samples_split 2
max_leaf_nodes 73.0
random_state 990.0
class_weight 0.08 0.96
-------------------------------------------------------

***1 번째 G_measure = 0.8444578417018345***

              precision    recall  f1-score   support

       False      0.988     0.770     0.866      1376
        True      0.369     0.934     0.529       198

    accuracy                          0.791      1574
   macro avg      0.679     0.852     0.697      1574
weighted avg      0.910     0.791     0.823      1574

[[1060  316]
 [  13  185]]
--------------------------2 번째 Train--------------------------
Initilialize Poplulation------> Complete



-------------------------------------------------------
Best parameters are:
criterion gini
max_depth 13.0
min_samples_split 9
max_leaf_nodes 105.0
random_state 810.0
class_weight 0.09 0.8
-------------------------------------------------------

***2 번째 G_measure = 0.7875980042765501***

              precision    recall  f1-score   support

       False      0.961     0.803     0.875      1376
        True      0.361     0.773     0.492       198

    accuracy                          0.799      1574
   macro avg      0.661     0.788     0.683      1574
weighted avg      0.885     0.799     0.827      1574

[[2165  587]
 [  58  338]]
--------------------------3 번째 Train--------------------------
Initilialize Poplulation------> Complete
